<a href="https://colab.research.google.com/github/knoppin1/DATA-690-WANG/blob/master/Assignment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 06 - Using Pandas
### Ken Noppinger (knoppin1@umbc.edu)
## Instructions
- This assignment is a re-do of assignment 04. Use Pandas and functions instead.
- Use the simple and convenient read_html() from Pandas to extract **HTML tables** from web pages. 
- Sample solution is here:
- https://github.com/wcj365/python-stats-dataviz/blob/master/pandas_apply.ipynb
- Make sure you user **Markdown cells** to document the process and its steps.
## Requirements
1. Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html
2. Find out how many HTML tables Pandas retrieves from the web page. Find out which dataframe contain the data and use it for further processing.
3. Display and explore the data (rows, columns, etc.) and determine the data quality (bad rows, bad columns, bad elements, null values, etc.)
4. Document the data quality issues using a Markdown cell so the reader understands what the problems are.
5. Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. Test the function by using test strings (for example, "1989*", "20*10")
6. Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.
7. Repeat this process for column "Total Population", 	"Census Cost", 	"Average Cost Per Person". (define, test, and apply the function)
8. After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.
9. Use Pandas to read the saved cleansed file and explore to make sure it is clean. 
10. Upload both the Jupyter notebook and the cleansed data file to GitHub and submit the GitHub link in BlackBoard.


---

In [1]:
import pandas as pd

---
## **1 -  Read HTML tables**
- Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html
-  Read all HTML tables into a list named **census_cost_data**. 



In [2]:
census_cost_data = pd.read_html("https://www.genealogybranches.com/censuscosts.html")
type(census_cost_data)

list

---
## **2 - Determine # of HTML tables Pandas retrieved from the web page.**

In [3]:
len(census_cost_data)

2

In [4]:
census_cost_data

[              0                 1               2                        3
 0   Census Year  Total Population     Census Cost  Average Cost Per Person
 1          1790           3929214         $44,377               1.13 cents
 2          1800           5308483         $66,109               1.24 cents
 3          1810           7239881        $178,445               2.46 cents
 4          1820           9633822        $208,526               2.16 cents
 5          1830          12866020        $378,545               2.94 cents
 6          1840          17069458        $833,371               4.88 cents
 7          1850          23191876      $1,423,351               6.14 cents
 8          1860          31443321      $1,969,377               6.26 cents
 9          1870          38558371      $3,421,198               8.87 cents
 10         1880          50155783      $5,790,678              11.54 cents
 11         1890          62979766     $11,547,127              18.33 cents
 12         

The census data is a list containing two dataframes.  The census data of interest is in the first datframe, so save that as **census_df**

In [5]:
census_df = census_cost_data[0]
census_df.head(5)

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


First row contains column headers, so make that row the header for the dataframe

In [6]:
census_df.columns = census_df.iloc[0]
census_df.drop([0], inplace=True)
census_df.reset_index(drop=True, inplace=True)
census_df.head(5)

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,1790,3929214,"$44,377",1.13 cents
1,1800,5308483,"$66,109",1.24 cents
2,1810,7239881,"$178,445",2.46 cents
3,1820,9633822,"$208,526",2.16 cents
4,1830,12866020,"$378,545",2.94 cents


---
## **3 - Explore the data** 

### Number of rows and columns

In [7]:
census_df.shape

(23, 4)

The census data has 23 rows and 4 columns

### Check dataframe for nulls

Reference - https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe

In [8]:
census_df.isnull().values.any()

False

Dataframe doesn't contain any nulls requiring cleanup.

### Check dataframe columns and rows for bad data

In [9]:
census_df.dtypes

0
Census Year                object
Total Population           object
Census Cost                object
Average Cost Per Person    object
dtype: object

Note the columns are all objects.  They will need to be cleaned up and cast to proper data dtypes.

In [10]:
census_df.describe()

,Census Year,Total Population,Census Cost,Average Cost Per Person
count,23,23,23,23
unique,23,23,23,23
top,2000,62979766,"$11,854,000",15.54 cents
freq,1,1,1,1


#### Inspect Census Year column

In [11]:
census_df['Census Year']

0      1790
1      1800
2      1810
3      1820
4      1830
5      1840
6      1850
7      1860
8      1870
9      1880
10     1890
11     1900
12     1910
13     1920
14     1930
15     1940
16     1950
17     1960
18     1970
19     1980
20     1990
21     2000
22    2010*
Name: Census Year, dtype: object

Note that the row at index 22 has an asterisk in the year value.  This will need to be cleaned by stripping the asterisks from values and then should be re-cast as integer type.

#### Inspect Total Population column

In [12]:
census_df['Total Population']

0       3929214
1       5308483
2       7239881
3       9633822
4      12866020
5      17069458
6      23191876
7      31443321
8      38558371
9      50155783
10     62979766
11     76303387
12     91972266
13    105710620
14    122775046
15    131669275
16    151325798
17    179323175
18    203302031
19    226542199
20    248718301
21    281421906
22    308745538
Name: Total Population, dtype: object

All Total Population values are integers.  Cleaning will not be required, but the column should be re-cast to integer type.

#### Inspect Census Cost column

In [13]:
census_df['Census Cost']

0            $44,377
1            $66,109
2           $178,445
3           $208,526
4           $378,545
5           $833,371
6         $1,423,351
7         $1,969,377
8         $3,421,198
9         $5,790,678
10       $11,547,127
11       $11,854,000
12       $15,968,000
13       $25,117,000
14       $40,156,000
15       $67,527,000
16       $91,462,000
17      $127,934,000
18      $247,653,000
19    $1,078,488,000
20    $2,492,830,000
21      $4.5 Billion
22       $13 Billion
Name: Census Cost, dtype: object

Census cost data contains dollar and comma characters as well as the text string "Billon".  The characters need to be stripped out and the "Billion" text needs to be applied to the number.  The column should be re-cast as integer type as well. 

#### Inspect Average Cost Per Person column

In [14]:
census_df['Average Cost Per Person']

0      1.13 cents
1      1.24 cents
2      2.46 cents
3      2.16 cents
4      2.94 cents
5      4.88 cents
6      6.14 cents
7      6.26 cents
8      8.87 cents
9     11.54 cents
10    18.33 cents
11    15.54 cents
12    17.07 cents
13    23.76 cents
14    32.71 cents
15    51.29 cents
16    60.44 cents
17    71.34 cents
18          $1.22
19          $4.76
20         $10.02
21         $15.99
22         $42.11
Name: Average Cost Per Person, dtype: object

Average Cost Per Person data contains dollar and decimal characters and the text string "cents".  The dollar characters need to be stripped out. Replace "cents" by removing the word and dividing cost by 100.  


---
## **4 - Data quality issues summarized.**

- **Census Year** was reaqd in as an object type due to characters in the values for the year.  This column requires cleansing and a re-casting to integer type.

- **Total Population** was read in as an object type, but all values were integers.  This column requires cleansing and a re-casting to integer type.

- **Census Cost** data contains dollar and comma characters as well as the text string "Billon".  The characters need to be stripped out and the "Billion" text needs to be applied to the number.  The column should be re-cast as integer type as well. 

- **Average Cost Per Person** data contains dollar and decimal characters and the text string "cents"  The dollar characters need to be stripped out. Replace "cents" by removing the word and dividing cost by 100.  Column data type should be float.

---

## **5 - cleanse_year()**

Function that takes a string as input and removes any asterisks in the string and returns the cleansed string. 

In [15]:
def cleanse_year(year_string):
    return year_string.strip('*')

Test the function by using test strings (for example, "1989*", "20*10")

In [16]:
cleanse_year("1989*")

'1989'

In [17]:
cleanse_year("*20*10")

'20*10'


## **6 - Cleanse "Census Year" column using cleanse_year() function and Pandas's apply() function.**


In [18]:
census_df["Clean Census Year"] = census_df["Census Year"].apply(cleanse_year).astype(int)
census_df

,Census Year,Total Population,Census Cost,Average Cost Per Person,Clean Census Year
0,1790,3929214,"$44,377",1.13 cents,1790
1,1800,5308483,"$66,109",1.24 cents,1800
2,1810,7239881,"$178,445",2.46 cents,1810
3,1820,9633822,"$208,526",2.16 cents,1820
4,1830,12866020,"$378,545",2.94 cents,1830
5,1840,17069458,"$833,371",4.88 cents,1840
6,1850,23191876,"$1,423,351",6.14 cents,1850
7,1860,31443321,"$1,969,377",6.26 cents,1860
8,1870,38558371,"$3,421,198",8.87 cents,1870
9,1880,50155783,"$5,790,678",11.54 cents,1880


In [19]:
# Verify "Clean Census Year" has integer data type
census_df["Clean Census Year"].dtype

dtype('int64')

## **7 - Create cleansing functions for "Total Population", "Census Cost", & "Average Cost Per Person".**

Define, test, and apply each function

### Clean Total Population

In [20]:
def remove_non_numerics(s):
    """ remove_non_numerics - removes all non-numeric characters and returns just the integers from the input string."""
    return ''.join(c for c in s if c.isdigit())

In [21]:
def cleanse_total_population(population_string):
    # This function serves as a hook to add cleansing steps for population strings.
    return remove_non_numerics(population_string)

In [22]:
# Test cleanse_total_population function
cleanse_total_population("98,0008ads98f**")

'98000898'

In [23]:
# Apply cleanse_total_population function.  Note no cleansing is necessary on the census data sample, but run through filter to remove non-numeric characters anyway
census_df["Clean Total Population"] = census_df["Total Population"].apply(cleanse_total_population).astype(int)
census_df

,Census Year,Total Population,Census Cost,Average Cost Per Person,Clean Census Year,Clean Total Population
0,1790,3929214,"$44,377",1.13 cents,1790,3929214
1,1800,5308483,"$66,109",1.24 cents,1800,5308483
2,1810,7239881,"$178,445",2.46 cents,1810,7239881
3,1820,9633822,"$208,526",2.16 cents,1820,9633822
4,1830,12866020,"$378,545",2.94 cents,1830,12866020
5,1840,17069458,"$833,371",4.88 cents,1840,17069458
6,1850,23191876,"$1,423,351",6.14 cents,1850,23191876
7,1860,31443321,"$1,969,377",6.26 cents,1860,31443321
8,1870,38558371,"$3,421,198",8.87 cents,1870,38558371
9,1880,50155783,"$5,790,678",11.54 cents,1880,50155783


In [24]:
# Verify "Clean Total Population" has integer data type
census_df["Clean Total Population"].dtype

dtype('int64')

### Clean Census Cost

In [25]:
def cleanse_census_cost(cost_string):

    # Remove commas and dollar signs
    clean_cost = cost_string.replace(",","").replace("$","")

    # Replace "Billion" with appropriate zeros
    if "Billion" in clean_cost.split():
        clean_cost = clean_cost.replace("Billion","")
        clean_cost = str(int(float(clean_cost)*1000000000))

    return clean_cost

In [26]:
# Test cleanse_census_cost function
cleanse_census_cost("$2,492,830,000")

'2492830000'

In [27]:
# Test cleanse_census_cost function
cleanse_census_cost("$4.5 Billion")

'4500000000'

In [28]:
# Apply cleanse_census_cost function.  
census_df["Clean Census Cost"] = census_df["Census Cost"].apply(cleanse_census_cost).astype(int)
census_df

,Census Year,Total Population,Census Cost,Average Cost Per Person,Clean Census Year,Clean Total Population,Clean Census Cost
0,1790,3929214,"$44,377",1.13 cents,1790,3929214,44377
1,1800,5308483,"$66,109",1.24 cents,1800,5308483,66109
2,1810,7239881,"$178,445",2.46 cents,1810,7239881,178445
3,1820,9633822,"$208,526",2.16 cents,1820,9633822,208526
4,1830,12866020,"$378,545",2.94 cents,1830,12866020,378545
5,1840,17069458,"$833,371",4.88 cents,1840,17069458,833371
6,1850,23191876,"$1,423,351",6.14 cents,1850,23191876,1423351
7,1860,31443321,"$1,969,377",6.26 cents,1860,31443321,1969377
8,1870,38558371,"$3,421,198",8.87 cents,1870,38558371,3421198
9,1880,50155783,"$5,790,678",11.54 cents,1880,50155783,5790678


In [29]:
# Verify Clean Census Cost has integer data type
census_df["Clean Census Cost"].dtype

dtype('int64')

### Clean Average Cost Per Person

In [30]:
def cleanse_avg_cost_per_person(cost_string):

    # Remove dollar signs.
    clean_cost = cost_string.replace("$","")

    # Replace "cents" by removing the word and dividing cost by 100
    if "cents" in clean_cost.split():
        clean_cost = clean_cost.replace("cents","")
        clean_cost = str("{:.4f}".format(float(clean_cost)/100))

    return clean_cost

In [31]:
# Test cleanse_avg_cost_per_person function
cleanse_avg_cost_per_person("71.34 cents")

'0.7134'

In [32]:
# Test cleanse_avg_cost_per_person function
cleanse_avg_cost_per_person("$42.11")

'42.11'

In [33]:
# Apply cleanse_avg_cost_per_person function.  
census_df["Clean Average Cost Per Person"] = census_df["Average Cost Per Person"].apply(cleanse_avg_cost_per_person).astype(float)
census_df

,Census Year,Total Population,Census Cost,Average Cost Per Person,Clean Census Year,Clean Total Population,Clean Census Cost,Clean Average Cost Per Person
0,1790,3929214,"$44,377",1.13 cents,1790,3929214,44377,0.0113
1,1800,5308483,"$66,109",1.24 cents,1800,5308483,66109,0.0124
2,1810,7239881,"$178,445",2.46 cents,1810,7239881,178445,0.0246
3,1820,9633822,"$208,526",2.16 cents,1820,9633822,208526,0.0216
4,1830,12866020,"$378,545",2.94 cents,1830,12866020,378545,0.0294
5,1840,17069458,"$833,371",4.88 cents,1840,17069458,833371,0.0488
6,1850,23191876,"$1,423,351",6.14 cents,1850,23191876,1423351,0.0614
7,1860,31443321,"$1,969,377",6.26 cents,1860,31443321,1969377,0.0626
8,1870,38558371,"$3,421,198",8.87 cents,1870,38558371,3421198,0.0887
9,1880,50155783,"$5,790,678",11.54 cents,1880,50155783,5790678,0.1154


In [34]:
# Verify "Clean Average Cost Per Person" has float data type
census_df["Clean Average Cost Per Person"].dtype

dtype('float64')

### Create cleaned dataframe with just the clean data

In [35]:
clean_census_df = census_df.drop(["Census Year", "Total Population", "Census Cost", "Average Cost Per Person"], axis=1)
clean_census_df.columns = ["Census Year", "Total Population", "Census Cost", "Average Cost Per Person"]
clean_census_df

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,1790,3929214,44377,0.0113
1,1800,5308483,66109,0.0124
2,1810,7239881,178445,0.0246
3,1820,9633822,208526,0.0216
4,1830,12866020,378545,0.0294
5,1840,17069458,833371,0.0488
6,1850,23191876,1423351,0.0614
7,1860,31443321,1969377,0.0626
8,1870,38558371,3421198,0.0887
9,1880,50155783,5790678,0.1154


In [36]:
# Verify data types of cleaned dataframe
clean_census_df.dtypes

Census Year                  int64
Total Population             int64
Census Cost                  int64
Average Cost Per Person    float64
dtype: object

## **8 - Save cleansed dataframe to a file named "census_cost_cleansed.csv" using CSV format.**

In [37]:
clean_census_df.to_csv("census_cost_cleansed.csv", index=False)

## **9 - Use Pandas to read the saved cleansed file and explore to make sure it is clean.**

In [38]:
df = pd.read_csv("census_cost_cleansed.csv")
df.shape

(23, 4)

In [39]:
df

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,1790,3929214,44377,0.0113
1,1800,5308483,66109,0.0124
2,1810,7239881,178445,0.0246
3,1820,9633822,208526,0.0216
4,1830,12866020,378545,0.0294
5,1840,17069458,833371,0.0488
6,1850,23191876,1423351,0.0614
7,1860,31443321,1969377,0.0626
8,1870,38558371,3421198,0.0887
9,1880,50155783,5790678,0.1154


In [40]:
# Verify data types of dataframe
df.dtypes

Census Year                  int64
Total Population             int64
Census Cost                  int64
Average Cost Per Person    float64
dtype: object

# COMPLETE